# Thesis work--2
# A new method for multivariate regression problem for improving fitness in Genetic Programming

### Importing the libraries

In [101]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from random import randrange, uniform
from sklearn.utils import check_random_state

### Global variables and functions for tuning

In [102]:
FEATURES = 3
NUMBER_OF_GENERATION = 20
ROWS = 300
NUMBER_OF_REGIONS = 3
formula = lambda X: X[:, 0]**2 - X[:, 1]**2 + X[:, 1] - 1

### Generating Training Samples

In [103]:
rng = check_random_state(0)
X = rng.uniform(-1, 1, ROWS).reshape(ROWS//FEATURES, FEATURES)
Y = formula(X)

### Splitting the dataset into the Training set and Test set

In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

### Applying PCA

In [105]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 1)
#print(X_train)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_
#print(X_train_pca)
#print(X_test_pca)
# print(np.shape(X_train),np.shape(X_test))
#print(explained_variance)

### Calculating DV

In [106]:
# sort X_train_pca and y_train by index
sorted_indexes = np.argsort(X_train_pca,axis=0)
sorted_x_train_pca = X_train_pca[sorted_indexes]
sorted_y_train = y_train[sorted_indexes]

# Finding Change of Slope
slope1 = []
slope2 = []
for itr in range(1,len(sorted_x_train_pca)):
    slope1.append((sorted_y_train[itr]-sorted_y_train[itr-1])/(sorted_x_train_pca[itr]-sorted_x_train_pca[itr-1]))
for itr in range(1,len(slope1)):
    slope2.append((slope1[itr]-slope1[itr-1])/(sorted_x_train_pca[itr]-sorted_x_train_pca[itr-1]))

# normalize slope2 
normalized_slope2 = (slope2-min(slope2))/(max(slope2)-min(slope2))

# Calculating Quantiles
normalized_slope2 = np.reshape(normalized_slope2,len(normalized_slope2))
quantile_ranges = pd.qcut(normalized_slope2,NUMBER_OF_REGIONS,labels=False,retbins=True)
quantile_ranges = quantile_ranges[1]

# Adding the difficult Vectors
difficult_points = {}
for q_ind in range(NUMBER_OF_REGIONS):
    low = quantile_ranges[q_ind]
    high = quantile_ranges[q_ind+1]
    difficult_points[q_ind] = []
    for n_ind in range(0,len(normalized_slope2)):
        if normalized_slope2[n_ind] >= low and normalized_slope2[n_ind] <= high:
            difficult_points[q_ind].append(n_ind)
print(difficult_points)

{0: [6, 8, 11, 15, 19, 21, 24, 28, 30, 31, 33, 35, 38, 40, 42, 45, 47, 52, 53, 58, 60, 61, 63, 66, 68, 70], 1: [0, 1, 2, 3, 5, 10, 12, 13, 14, 17, 18, 22, 26, 29, 34, 44, 49, 50, 56, 65, 67, 72, 73, 74, 75, 77], 2: [4, 7, 9, 16, 20, 23, 25, 27, 32, 36, 37, 39, 41, 43, 46, 48, 51, 54, 55, 57, 59, 62, 64, 69, 71, 76]}


### Training of converted PCA

In [107]:
from gplearn.genetic import SymbolicRegressor
est_gp = SymbolicRegressor(population_size=200,
                           generations=NUMBER_OF_GENERATION, stopping_criteria=0.01,
                           p_crossover=0.7, p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, p_point_mutation=0.1,
                           max_samples=0.9, verbose=1,
                           parsimony_coefficient=0.01, random_state=0)


### Getting the different regions of difficulty level

In [108]:
# Hard to evolve points
hard_to_evolve_x = X_train_pca[difficult_points[0]]
hard_to_evolve_y = y_train[difficult_points[0]]

# Medium to evolve points
medium_to_evolve_x = X_train_pca[difficult_points[1]]
medium_to_evolve_y = y_train[difficult_points[1]]

# Easy to evolve points
easy_to_evolve_x = X_train_pca[difficult_points[2]]
easy_to_evolve_y = y_train[difficult_points[2]]


### Feed the system hard to evolve points first

In [109]:
est_gp.fit(hard_to_evolve_x,hard_to_evolve_y)
print(est_gp._program)

    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    46.36 1154.6476506965757        7 0.5098817769016858 0.6436402628820348      4.40s
   1    12.87 1.4143760780885948        7 0.39180868401004604 0.44834529979141063      4.92s
   2     6.76 1.8874227552499951       11 0.37719243611499786 0.6512415386797877      4.16s
   3     4.59 1.4350434117059985        7 0.3541753496319566 0.7368675300234296      3.91s
   4     3.58 0.7468164095711978        7 0.3559609111867784 0.7231782247697959      3.75s
   5     4.36 0.8830114420639997        7 0.3489873191542264 0.7766424303526945      3.61s
   6     5.34 0.9034748582289555        7 0.3426612835153983 0.8251420369170438      3.40s
   7     4.67 0.8345713580456625        7 0.34609587985472784 0.7407412310015024      3.15s
   8     5.95 1.1138480

### Feed the system Medium to evolve points keeping the current modal state appending generation by current generation

In [110]:
est_gp.set_params(generations=NUMBER_OF_GENERATION*2, warm_start=True)
est_gp.fit(medium_to_evolve_x,medium_to_evolve_y)
print(est_gp._program)

    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
  20     7.37 1.1746799715654015        7 0.34752631889062735 0.4622070937575978      0.27s
  21     6.75 1.3406986607649671        5 0.31791029797129733 0.6112042110184154      0.52s
  22     6.43 1.1882428096154083        5 0.2945751645211446 0.7901069008029199      0.67s
  23     5.74 1.2575832657426063        5 0.28043978633254607 0.8984781335821745      0.76s
  24     5.91 1.4479615958534893        5 0.2950891600074032 0.7861662687416034      0.80s
  25     5.46 1.0999330365311517        5 0.28738153863405197 0.8452580326039629      0.83s
  26     5.14 1.2435470733790563        5 0.2751704793576012 0.938876153723419      0.84s
  27     5.05 1.1897187426904048        5 0.2915066677816123 0.813632042472667      0.83s
  28     5.34 2.140192021

### Same process but for easy to evolve points and this is our result

In [111]:
est_gp.set_params(generations=NUMBER_OF_GENERATION*3, warm_start=True)
est_gp.fit(easy_to_evolve_x,easy_to_evolve_y)
print(est_gp._program)

    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
  40     5.05 1.1623666809134192        5 0.37160724180542426 0.9045108093578799      0.12s
  41     5.19 1.2206754446208559        5 0.35180407017037657 1.0563351252265776      0.20s
  42     5.66 17.768400952326747        5 0.3695384616369878 0.9203714573158926      0.28s
  43     5.28 1.419093861134413        5 0.36818421907277693 0.9307539836415085      0.32s
  44     6.12 13.501130655170405        5 0.3779298357949252 0.8560375887717065      0.38s
  45     4.69 1.2445755711124526        5 0.36739691615730996 0.9367899726600898      0.40s
  46     5.31 1.1822298770796977        5 0.36771673392674764 0.9343380364277337      0.43s
  47     5.24 1.1131919852356962        5 0.36214949939870267 0.9770201678094118      0.43s
  48     5.11 1.137958

### TODO check the results accuracy and compare this method with Standard GP make graphs for visualization